In [192]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier

In [2]:
df=pd.DataFrame(pd.read_csv("linear_train[1].txt",header=None))

In [21]:
a=df[0].apply(len)

In [22]:
df['len']=a

In [48]:
vowel=['е','о','у','а','э','я','и','ю']

In [58]:
df['num_vow']=df[0].apply(lambda x: sum(1 for i in x if i.lower() in vowel)).astype(int)

In [60]:
df['num_unvowel']=df['len']-df['num_vow']

In [61]:
df.head()

,0,1,len,num_vow,num_unvowel
0,Аалтонен,1,8,4,4
1,Аар,0,3,2,1
2,Аарон,0,5,3,2
3,ААРОН,0,5,3,2
4,Аарона,0,6,4,2


In [86]:
fam=['ин','ов','ова','ина','ев','ева','ых','кий','кая','ич']
df['fam']=df[0].apply(lambda x: any(x.lower().endswith(hui) for hui in fam)).astype(int)

In [88]:
df.head()

,0,1,len,num_vow,num_unvowel,fam
0,Аалтонен,1,8,4,4,0
1,Аар,0,3,2,1,0
2,Аарон,0,5,3,2,0
3,ААРОН,0,5,3,2,0
4,Аарона,0,6,4,2,0


In [143]:
np.mean(cross_val_score(a, X_test, y_test, cv=5, scoring='roc_auc'))

0.680688329897386

In [135]:
f=['б', 'в', 'г', 'д', 'ж', 'з', 'й', 'к', 'л', 'м', 'н', 'п', 'р', 'с', 'т', 'ф', 'х', 'ч', 'ц', 'ш', 'щ']
def max_coun(w):
    k=0
    l=[0]
    for i in w:
        if i.lower() in f:
            k+=1
        else:
            l.append(k)
            k=0
    l.append(k)
    m=max(l) 
    return m
    """if m>3:
        return 0
    else:
        return 1"""

In [141]:
vowel=['е','о','у','а','э','я','и','ю']
def max_vow(w):
    k=0
    l=[0]
    for i in w:
        if i.lower() in vowel:
            k+=1
        else:
            l.append(k)
            k=0
    l.append(k)
    m=max(l) 
    return m

In [166]:
def to_normal(df1):
    df=df1.copy()
    
    vowel=['е','о','у','а','э','я','и','ю']
    df['len']=df[0].apply(len)
    df['num_vow']=df[0].apply(lambda x: sum(1 for i in x if i.lower() in vowel)).astype(int)
    df['num_unvowel']=df['len']-df['num_vow']
    fam=['ин','ов','ова','ина','ев','ева','ых','кий','кая','ич','ко','ая','ой','дзе','ым','у','им','ян','ук','юк']
    df['fam']=df[0].apply(lambda x: any(x.lower().endswith(hui) for hui in fam)).astype(int)
    df['Is_title']=df[0].apply(lambda x: x.istitle()).astype(int)
    df['c_in_a_row']=df[0].apply(lambda x: max_coun(x)).astype(int)
    #df['v_in_a_row']=df[0].apply(lambda x: max_vow(x)).astype(int)
    df['fam_len']=((df[0].apply(len))<3).astype(int)
    df['vow_per']=(df['num_vow']<=4).astype(int)
    alphabet=['а','б','в','г','д','е','ж','з','и','й','к','л','м','н','о','п','р','с','т','у','ф','х','ц','ч','ш','щ','ъ','ы','ь','э','ю','я']
    for j in alphabet:
        df[j]=df[0].apply(lambda x: sum(1 for i in x.lower() if i==j))
    #df[0]=df1
    return df

In [167]:
tmp=to_normal(df)

In [168]:
tmp.head()

,0,1,len,num_vow,num_unvowel,fam,Is_title,c_in_a_row,fam_len,vow_per,...,ц,ч,ш,щ,ъ,ы,ь,э,ю,я
0,Аалтонен,1,8,4,4,0,1,2,0,1,...,0,0,0,0,0,0,0,0,0,0
1,Аар,0,3,2,1,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,Аарон,0,5,3,2,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
3,ААРОН,0,5,3,2,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,Аарона,0,6,4,2,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0


In [105]:
param_grid = {'max_depth':[x for x in range(2,25)]}

In [115]:
grid=GridSearchCV(
   DecisionTreeClassifier(),  param_grid, n_jobs=2,scoring='roc_auc', cv=5
).fit(X_data, y_data)

In [119]:
grid.best_params_

{'class_weight': None,
 'criterion': 'gini',
 'max_depth': 8,
 'random_state': 42,
 'splitter': 'random'}

In [169]:
clf=DecisionTreeClassifier(**grid.best_params_)
X_data=tmp.drop(labels=[0,1],axis=1)
y_data=tmp[1]
np.mean(cross_val_score(clf, X_data, y_data, cv=5, scoring='roc_auc'))

0.8194618378876672

In [170]:
catbost=CatBoostClassifier(cat_features=['fam','Is_title'])
np.mean(cross_val_score(clf, X_data, y_data, cv=5, scoring='roc_auc'))

0.8194618378876672

In [196]:
rndmfrst=RandomForestClassifier()
np.mean(cross_val_score(rndmfrst, X_data, y_data, cv=5, scoring='roc_auc'))

0.7655452596094002

In [ ]:
param_grid1 = {'num_leaves':[x for x in range(30,150)],'learning_rate':[0.01*x for x in range(2,12)],'n_estimators':[x for x in range(100,250)]}
grid1=GridSearchCV(
   LGBMClassifier(),  param_grid1, n_jobs= -1,scoring='roc_auc', cv=5
).fit(X_data, y_data) 

In [217]:
lgbt=LGBMClassifier(num_leaves=120,learning_rate=0.08,n_estimators=150)
np.mean(cross_val_score(lgbt, X_data, y_data, cv=5, scoring='roc_auc'))

0.8300296924769391

In [193]:
clf.fit(X_data,y_data)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=8,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='random')

In [198]:
lgbt.fit(X_data,y_data)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [199]:
test=pd.DataFrame(pd.read_csv('linear_test[1].txt',header=None))

In [200]:
normal_test=to_normal(test)

In [201]:
normal_test.head()

,0,len,num_vow,num_unvowel,fam,Is_title,c_in_a_row,fam_len,vow_per,а,...,ц,ч,ш,щ,ъ,ы,ь,э,ю,я
0,Аалто,5,3,2,0,1,2,0,1,2,...,0,0,0,0,0,0,0,0,0,0
1,ААР,3,2,1,0,0,1,0,1,2,...,0,0,0,0,0,0,0,0,0,0
2,Аара,4,3,1,0,1,1,0,1,3,...,0,0,0,0,0,0,0,0,0,0
3,Ааре,4,3,1,0,1,1,0,1,2,...,0,0,0,0,0,0,0,0,0,0
4,Аарон,5,3,2,0,1,1,0,1,2,...,0,0,0,0,0,0,0,0,0,0


In [202]:
nms=normal_test[0]
pudge=normal_test.drop(labels=0,axis=1)

In [209]:
predictions=lgbt.predict_proba(pudge)[:,1]

In [210]:
itog=pd.DataFrame(nms)

In [211]:
itog['Id']=np.arange(len(itog))
itog['Answer']=predictions
itog=itog.drop(labels=0,axis=1)

In [212]:
itog.head()

,Id,Answer
0,0,0.253858
1,1,0.082219
2,2,0.247291
3,3,0.318857
4,4,0.304977


In [214]:
itog.to_csv("output228_lgbm.txt",index=False)

In [ ]:
заглавные
endswith
согласные